# NCBI BLAST

In [ ]:
import csv
from Bio import SeqIO
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

In [ ]:
def run_blast(protein_accession):
    # Retrieve protein sequence using accession ID
    handle = NCBIWWW.qblast("blastp", "nr", protein_accession)

    # Save the BLAST results to a file
    with open("blast_results.xml", "w") as output_file:
        output_file.write(handle.read())

    # Parse the BLAST results
    result_handle = open("blast_results.xml")
    blast_records = NCBIXML.parse(result_handle)
    blast_record = next(blast_records)

    # Create a list to store the results
    results = []

    # Process the BLAST results
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            sequence = alignment.title
            length = alignment.length
            evalue = hsp.expect
            score = hsp.score
            alignment_info = f"{hsp.align_length} matches, {hsp.identities} identities"

            # Calculate percentage similarity
            alignment_length = float(hsp.align_length)
            num_identities = hsp.identities
            percentage_similarity = (num_identities / alignment_length) * 100

            # Retrieve percentage identity
            percentage_identity = (num_identities / length) * 100

            # Retrieve accession
            accession = alignment.accession

            # Append the result to the list
            results.append([sequence, length, evalue, score, alignment_info, percentage_similarity, percentage_identity, accession])

    # Save the results to a CSV file
    with open("blast_results.csv", mode="w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Sequence", "Length", "E-value", "Score", "Alignment", "Percentage Similarity", "Percentage Identity", "Accession"])
        writer.writerows(results)

    result_handle.close()

# Run the BLAST search for a protein accession ID
protein_accession_id = "Q6FW99"
run_blast(protein_accession_id)

In [ ]:
NCBI BLAST for multiple proteins at once, on a loop

In [ ]:
def run_blast(protein_accession):
    # Retrieve protein sequence using accession ID
    handle = NCBIWWW.qblast("blastp", "nr", protein_accession)

    # Save the BLAST results to a file
    with open("blast_results.xml", "w") as output_file:
        output_file.write(handle.read())

    # Parse the BLAST results
    result_handle = open("blast_results.xml")
    blast_records = NCBIXML.parse(result_handle)
    blast_record = next(blast_records)

    # Create a list to store the results
    results = []

    # Process the BLAST results
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            sequence = alignment.title
            length = alignment.length
            evalue = hsp.expect
            score = hsp.score
            alignment_info = f"{hsp.align_length} matches, {hsp.identities} identities"

            # Calculate percentage similarity
            alignment_length = float(hsp.align_length)
            num_identities = hsp.identities
            percentage_similarity = (num_identities / alignment_length) * 100

            # Calculate percentage identity
            percentage_identity = (num_identities / length) * 100
            
            # Retrieve organism name
            organism = alignment.hit_def.split("[")[1].split("]")[0]

            # Retrieve accession
            accession_id = alignment.accession

            # Append the result to the list
            results.append([protein_accession, accession_id, sequence, length, evalue, score, alignment_info, percentage_similarity, percentage_identity, organism])

    result_handle.close()

    return results

# List of protein accession IDs
protein_accession_ids = ["P69905", "Q9H3D4", "P06634", "P39943", "P06185", "P40061", "P04908", "P38977", "P39951", "P39948"]

# Run BLAST for each accession ID
all_results = []
for accession_id in protein_accession_ids:
    print(f"Running BLAST for {accession_id}...")
    results = run_blast(accession_id)
    all_results.extend(results)

# Save all the results to a CSV file
with open("multiple_blast_results.csv", mode="w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["protein_accession", "Accession ID", "Sequence", "Length", "E-value", "Score", "Alignment", "Percentage Identity", "Organism"])
    writer.writerows(all_results)
